In [1]:
import cv2
import os
import json
import numpy as np
from tqdm import tqdm

# 讀取class_bgr.json文件
with open('class_bgr.json', 'r') as file:
    class_bgr = json.load(file)

# 建立從BGR值到類別名的映射
bgr_to_class = {tuple(bgr): class_name for class_name, bgr in class_bgr.items()}

# 準備文件路徑
inference_output_dir = './inference_data/ground_truth'

# 存儲每張圖像的類別名
documents = []

# 轉換並建構documents
for inference_filename in tqdm(os.listdir(inference_output_dir), desc="Processing images"):
    inference_path = os.path.join(inference_output_dir, inference_filename)
    segmentation_result = cv2.imread(inference_path)

    # 獲得該圖像的所有像素點的Class
    class_names = [bgr_to_class.get(tuple(segmentation_result[y, x].tolist()), "unknown") for y in range(segmentation_result.shape[0]) for x in range(segmentation_result.shape[1])]

    # 將類別名作為“文檔”
    documents.append(class_names)


Processing images: 100%|██████████| 242/242 [00:57<00:00,  4.23it/s]


In [2]:
# 建立詞彙表和索引
vocab = set(word for doc in documents for word in doc)
vocab_index = {word: i for i, word in enumerate(vocab)}

# 手動計算TF
tf_matrix = np.zeros((len(documents), len(vocab)))

for doc_idx, doc in enumerate(documents):
    for word in doc:
        word_idx = vocab_index[word]
        tf_matrix[doc_idx, word_idx] += 1

# 計算每個類別出現在文檔的比例
class_presence = np.count_nonzero(tf_matrix, axis=0) / len(documents)

# # 對TF矩陣中的每個類別按其出現比例進行加權
# tf_weighted = tf_matrix * class_presence


# 手動計算IDF
n_documents = len(documents)
df = np.count_nonzero(tf_matrix, axis=0)
# idf = np.log((n_documents + 1e-9) / (df + 1e-9))

In [5]:
print(len(tf_matrix))
print(len(tf_matrix[0]))
# pixel_probability_matrix = tf_matrix/(1280*720)
# weights = 1 / (pixel_probability_matrix + 1e-9)
print(class_presence)
print(tf_matrix[0])
print(len(documents))
# print(tf_weighted[0].astype(int))

242
10
[0.80991736 0.39256198 0.7768595  0.92561983 1.         0.20247934
 0.47933884 0.56198347 0.98760331 0.13636364]
[ 28165.      0.  14736. 326573.  22715.      0.      0.   8232. 521179.
      0.]
242


In [9]:
df = np.count_nonzero(tf_matrix, axis=0)/n_documents
df

array([0.80991736, 0.39256198, 0.7768595 , 0.92561983, 1.        ,
       0.20247934, 0.47933884, 0.56198347, 0.98760331, 0.13636364])

In [14]:
# 對TF取倒數並與IDF相乘
tf_inverse = 1 / (tf_matrix + 1e-9)  # 避免除以0
weights = tf_inverse * (df*df)

# or 取tf倒數作為權重
# tf_weighted = tf_weighted.astype(int)
# weights = ( 1 / (tf_matrix + 1e-9) ) * class_presence * class_presence

# 對權重進行歸一化
print("[INFO] Normalizing...")
weights_normalized = weights / np.linalg.norm(weights, axis=1, keepdims=True)

# 計算每個類別的權重並保存
class_weights = np.sum(weights_normalized, axis=0)
class_weights_dict = {word: weight for word, weight in zip(vocab, class_weights)}

[INFO] Normalizing...


In [15]:
with open('class_weight_woIDFver3.json', 'w') as json_file:
    json.dump(class_weights_dict, json_file, indent=4)

print("[INFO] Class weights have been saved.")

[INFO] Class weights have been saved.
